In [ ]:
#directions = [5,15,20,25,35]
#labels = ['extremeleft','left','middle','right','extremeright']

directions = [0,5,10,15,20,25,30,35]
N = len(directions)
labels = ['baseline','5','10','15','middle','25','30','extremeright']

labeldict = dict(zip(directions,labels))
n_each = 20 #15

In [ ]:
%pylab inline
import random
import pandas as pd
import numpy as np
import time
import multiprocessing
import ctypes
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import os

In [ ]:
def create_random_design():
    dirs = []
    lastdir = None
    for _ in range(n_each):
        acceptable = False
        while not acceptable:
            d = range(N) #directions[:]
            random.shuffle(d)
            acceptable = d[0]!=lastdir
        lastdir=d[-1]
        dirs+=d
    return dirs

In [ ]:
dirs = create_random_design()
" ".join([str(directions[d]) for d in dirs ])

In [ ]:
len(dirs)

## What are the carry-over effects?

In [ ]:
def get_transitions(dirs):
    # Get the first-order transition matrix
    trans = [ (dirs[i],dirs[i+1]) for i in range(len(dirs)-1) ]
    #n = len(directions)
    transcount = np.zeros((N,N))
    for i in range(len(dirs)-1):
        transcount[dirs[i],dirs[i+1]]+=1
    return transcount

In [ ]:
transcount = get_transitions(dirs)
transcount

In [ ]:
imshow(transcount)

In [ ]:
#sum(transcount,0)

In [ ]:
20./7

In [ ]:
3*6+1*2

## Batch-generate schedules and pick the best one
Generations of scholars will wonder why on Earth we do this, but hey...

In [ ]:
def badness(mat):
    # Tell us how bad this transition matrix is - how unequal are the transitional probability distributions?
    
    # Compute the average of each row
    mat = np.array(mat)
    n,_ = mat.shape
    # Restrict to the off-diagonal elements 
    rowvalues = [ vals[sel] for (vals,sel) in zip(mat,(1-np.eye(n)).astype(bool)) ]
    
    # Compute the deviation of each item from the average for that row
    dists = [ [ abs(r-np.mean(row)) for r in row ] for row in rowvalues ]
    
    return sum(np.array(dists).flatten())

In [ ]:
badness(transcount)

In [ ]:
#N_DESIGNS = 1000000 # Neeraj wanted 100000000
N_DESIGNS = 100000000
N_PARALLEL = 28

# Distribution of badnesses
shared_array_base = multiprocessing.Array(ctypes.c_double, N_DESIGNS)
shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
#shared_array = shared_array.reshape(10, 10)

# Best the design
shared_design_base = multiprocessing.Array(ctypes.c_int, n_each*len(directions))
shared_design = np.ctypeslib.as_array(shared_design_base.get_obj())

bestbad = multiprocessing.Value('d', 999999)


def generate_random(i,allbadness=shared_array,bestdesign=shared_design,bestb=bestbad):
    #print(i)
    design = create_random_design()
    trans = get_transitions(design)
    howbad = badness(trans)
    allbadness[i]=howbad
    if howbad<bestb.value:
        bestb.value=howbad
        for j,v in enumerate(design):
            shared_design[j]=v
    if i%100000==0:
        print("{} best {}".format(i,bestbad.value))
    return 

# Parallel processing
#def my_func(i, def_param=shared_array):
#    shared_array[i,:] = i

t0 = time.time()

pool = multiprocessing.Pool(processes=N_PARALLEL)
pool.map(generate_random, range(N_DESIGNS))

tsec = (time.time()-t0)
tdur = tsec/(60*60)

print("This took {} seconds i.e. {} hours for {} designs".format(tsec,tdur,N_DESIGNS))
targetdes = 100000000
print("At this rate {} will take {} hours.".format(targetdes,targetdes*tdur/N_DESIGNS))

#print(shared_array)
#bestbad_posthoc = min(shared_array)
#print(bestbad_posthoc,bestbad.value)

In [ ]:
sns.distplot(shared_array[:1000000])
axvline(x=bestbad.value,color='red')
sns.despine(offset=5)
print(bestbad.value)

In [ ]:
dirs = list(shared_design)

In [ ]:
transcount = get_transitions(dirs)
transcount

In [ ]:
imshow(transcount)

In [ ]:
# Let's save the history
import pickle
PICKLE_F = 'history.pickle'
if os.path.exists(PICKLE_F):
    history = pickle.load(open(PICKLE_F,'rb'))
else:
    history = []
history.append({"best_design":list(shared_design)})
pickle.dump(history,open(PICKLE_F,'wb'))

## Output

In [ ]:
chosen = [ directions[d] for d in shared_design ]
' '.join([str(s) for s in chosen])

In [ ]:
tab = pd.DataFrame({'direction':chosen})
tab['type']=[ labeldict[d] for d in chosen ]
tab['trial']=np.arange(len(chosen))+1
#tab.to_csv('recognitionyesno_theOne.csv')
tab.to_csv('recognitionyesno160.csv')

In [ ]:
tab

# Doing this the smart way (maybe)

In [ ]:
if True:
    #Highest number in square
    order_of_sq = int(input("Enter order of sq: "))

    #Number you want to start the square with
    top_left = int(input("Enter top left number: "))

    #Sets a placeholder for a variable called top_left_init
    top_left_init=0

    #Sets the initial value of top_left to a new variable because the code will change the value of top left later on 
    top_left_init += top_left

    #Initialize the value of count
    count = 0

    #Add 1 to the highest value in latin square to account for the range function (the ending number is always one less than the number you enter into the range function)
    for values in range (1,order_of_sq+1):

        #Prevents the program from adding too many characters to the line
        while count != order_of_sq:

            #Prints numbers with spaces after them in a horizontal manner
            print(top_left,sep=' ',end=' ')

            #Adds 1 to the top_left
            top_left += 1

            #Count is used to keep track of how many characters are in your line
            count+=1

            #Restarts the numbers in your line when you reach the highest number
            if top_left == order_of_sq+1:
                top_left = 1

        #Creates a new row
        print()
        count = 0

        #Calls the initial value of top_left and adds 1 to it before beginning the next row
        top_left_init += 1

        #Resets top_left_init to 1 if it reaches the highest number in the square
        if top_left_init == order_of_sq + 1:
            top_left_init = 1
            top_left = top_left_init
        else:
            top_left = top_left_init